# Setup

In [1]:
# mount drive 
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd "/content/gdrive/My Drive/Github/lc-classification"

/content/gdrive/My Drive/Github/lc-classification


In [3]:
import numpy as np
import pandas as pd
import pickle

!pip install transformers
import torch
from transformers import LongformerTokenizerFast

     |████████████████████████████████| 2.8 MB 3.2 MB/s 
     |████████████████████████████████| 3.3 MB 42.7 MB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
     |████████████████████████████████| 636 kB 57.2 MB/s 
     |████████████████████████████████| 895 kB 48.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Create datasets

In [4]:
# import fulldata
dataset = pd.read_json('./data/interim/gutenberg_dataset.json')

# import subclasses metadata
with open("./work/subclasses.txt", encoding='utf-8') as f:
    lines = f.readlines()
lc_subclasses = {}
for i in range(len(lines)):
    lc_subclasses[lines[i][0]] = lines[i][2:].replace(" ", "").replace("\n", "").split(",")

In [5]:
# push classified books to json
window = 4096
skip = 300
idx, X, y_class, y_subclass = [], [], [], []
for cls in lc_subclasses:
    for i in range(len(dataset)):
        for sub in dataset.subjects_new[i]:
            if sub in lc_subclasses[cls]:
                idx.append(dataset.id[i])
                tokens = ' '.join(dataset.text[i].split()[skip:4096+skip])
                X.append(tokens)
                y_class.append(cls) 
                y_subclass.append(sub)
fullset = pd.DataFrame({'id':idx, 'X':X, 'y_class':y_class, 'y_subclass':y_subclass})

In [6]:
fullset.head()

,id,X,y_class,y_subclass
0,200,the original publisher have been changed or de...,A,AE
1,247,"tone in the model major scale (that in c), or ...",A,AG
2,248,"appropriate (to); to refer, as an effect to a ...",A,AG
3,636,in its pursuit; that millions of people become...,A,AZ
4,713,or both combined. every one of these causes in...,A,AZ


In [7]:
fullset.groupby('y_class').count().id

y_class
A     2600
B     3641
C      315
D     4500
G     1030
H     1423
J      323
K      198
L      284
M      399
N      902
P    28158
Q     1936
R      486
S      659
T     1242
U      214
V       81
Z      459
Name: id, dtype: int64

# Train test split

In [8]:
# train test split 
np.random.seed(0)
msk1 = np.random.rand(len(fullset)) < 0.8
test_set = fullset.copy()[~msk1]
train_set = fullset.copy()[msk1]
np.random.seed(1)
msk2 = np.random.rand(len(train_set)) < 0.8
val_set = train_set.copy()[~msk2]
train_set = train_set.copy()[msk2]

# Sampling

In [9]:
train_sample = []
for cls in list(set(fullset.y_class)):
    length = train_set[train_set['y_class'] == cls].shape[0]
    if length >= 1000:
        sample = train_set[train_set['y_class'] == cls].sample(n=1000, replace=False, random_state=0)
    elif length < 1000:
        sample = train_set[train_set['y_class'] == cls].sample(n=max(length, 500), replace=(length<500), random_state=0)   
    train_sample.append(sample)

In [10]:
val_sample = []
for cls in list(set(fullset.y_class)):
    length = val_set[val_set['y_class'] == cls].shape[0]
    sample = val_set[val_set['y_class'] == cls].sample(n=min(length, 100), replace=False, random_state=0)
    val_sample.append(sample)

In [11]:
test_sample = []
for cls in list(set(fullset.y_class)):
    length = test_set[test_set['y_class'] == cls].shape[0]
    sample = test_set[test_set['y_class'] == cls].sample(n=min(length, 100), replace=False, random_state=0)
    test_sample.append(sample)

In [12]:
train_set = pd.concat(train_sample)
val_set = pd.concat(val_sample)
test_set = pd.concat(test_sample)
train_set.groupby('y_class').count().id

y_class
A    1000
B    1000
C     500
D    1000
G     655
H     919
J     500
K     500
L     500
M     500
N     569
P    1000
Q    1000
R     500
S     500
T     821
U     500
V     500
Z     500
Name: id, dtype: int64

In [13]:
fullset.shape, train_set.shape, val_set.shape, test_set.shape

((48850, 4), (12964, 4), (1475, 4), (1531, 4))

# Save dataframes

In [14]:
fullset.to_json('./data/full_set.json')
train_set.reset_index(drop=True, inplace=True)
train_set.to_json('./data/train_set.json')
val_set.to_json('./data/val_set.json')
test_set.to_json('./data/test_set.json')

# Save encodings and labels

In [15]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [16]:
# set parameters
MAX_LENGTH = 2048

# transform text to encodings
train_encodings = tokenizer(list(train_set.X), max_length=MAX_LENGTH, truncation=True, padding=True)
val_encodings = tokenizer(list(val_set.X), max_length=MAX_LENGTH, truncation=True, padding=True)
test_encodings = tokenizer(list(test_set.X), max_length=MAX_LENGTH, truncation=True, padding=True)

# save pickle files
def save_pickle(stuff, fileName):
    with open(fileName, 'wb') as f:
        pickle.dump(stuff, f, pickle.HIGHEST_PROTOCOL)

# saved encodings and labels 
save_pickle(train_encodings, './work/train_encodings.pkl')
save_pickle(val_encodings, './work/val_encodings.pkl')
save_pickle(test_encodings, './work/test_encodings.pkl')
save_pickle(train_set.y_class, './work/train_labels.pkl')
save_pickle(val_set.y_class, './work/val_labels.pkl')
save_pickle(test_set.y_class, './work/test_labels.pkl')